## Mechanical Tests

Mechanical tests check an implementation without regards for the science that underpins the code. This approach is often used when testing smaller chunks of code in isolation, e.g. unit-testing. The test can be given input that is as much physical nonsense, and return unphysical outputs, as long as the flow of data and data transformations are the same. Lets go directly to a simple example.

### Easy but unphysical input

When examining fluid flows, we are interested in the momentum $\mu$ at each lattice site. It can be obtained as the sum over the momentum of the particles at the site. However, lets promptly forget about the physics. Here is what the function looks like:

In [1]:
momentum(fᵢ::DenseVector, cᵢ::DenseMatrix) = cᵢ * fᵢ

momentum (generic function with 1 method)

We could try and setup physical inputs for this function and test the result. But it is much simpler to check using short integer inputs, rather than the longer floating points vectors and matrices the function will be given during an actual Lattice-Boltzmann calculation.

In [2]:
using FactCheck: @fact, facts
facts("Check momentum function over single site") do
    @fact momentum([1, 1], [1 2; 3 4]) => [3, 7]
    @fact momentum([-1, 1], [1 2; 3 4]) => [1, 1]
end
nothing

Check momentum function over single site
2 facts verified.


Sure, the `momentum` function is short and simple. But cutting the code down to size is what unit-testing is all about; it is to make testing (and coding) simpler that we adopt a coding style with short functions. We could copy that one line of code `cᵢ * fᵢ` and paste it everywhere `momentum` is used. Or we can make it a tested function, use it where the momentum is needed, and never have to worry about how momentum is computed anymore. And using input crafted for simplicity rather than physicality will get a lot of mileage in more complex situations as well.

### Comparison with (simplified) test algorithm

It is often possible to implement an algorithm several ways. For instance, when implementing the derivative of some function, it is a good idea to test it against the numerical derivatives, i.e. the comparison $\frac{\partial f(x)}{\partial x} \approx \frac{f(x+\delta x) - f(x)}{\delta x}$. Implementing a functionality twice might seem like a waste of time, but in practice it often proves invaluable. It is much easier to understand where a bug has crept in when there is an alternate implementation that can be queried for intermediate values (e.g. value of some variable `x` halfway through the code). It is true that during the implementation phase, when neither the target code nor the test are known to work, it is not always clear which is responsible for a discrepancy. But it is a price worth paying in practice.

This is how the function giving the fluid density $\rho$ at a site $\vec{r}$ is tested in LatBo. The density is defined simply as the sum over the number of particles at the site. The magic is that if the input that it is fed is a series, then we can use the result for a geometric series as a secondary algorithm $\sum_{i=1}^Ni = 0.5N(N+1)$: 

In [3]:
using FactCheck: @fact, facts

density(fᵢ::DenseVector) = sum(fᵢ)

facts("Check density by comparing with geometric series") do
    series(N) = N * (N+1)/2
    @fact density([1:10]) => series(10)
    @fact density([10:15]) => series(15) - series(9)
end
nothing

Check density by comparing with geometric series
2 facts verified.


We've done two things here: (i) the input is simplified to integers, (ii) the algorithm of `density` is compared to another, `series`. As always, one should be wary that the test covers all use cases, especially when comparing to a restricted algorithm as done here. Due to the simplicity of the original algorithm, there is little doubt in this case that the test is sufficient.

### Checking for specific behaviours

When testing, it is all too easy to convince one self that the numbers spewed by a piece of code are correct, when in fact they are garbage. Testing for behaviours instead is more robust. What kind of behaviour will depend on the actual method, but, in the case of mathematical functions, linearity is fairly common, at least for some arguments.

The introduction to Lattice-Boltzmann at the beginning of this chapter described the collision step where particles are exchanged between different populations. At least within the so-called _single-relaxation time_ approximation, this step is very simple:

In [4]:
collision(τ⁻¹::Number, fᵢ::DenseVector, feq::DenseVector) = τ⁻¹ * (feq - fᵢ)

collision (generic function with 1 method)

We can (and should) test for each argument separately. The function is linear with respect to $\tau^{-1}$ when the other variables are fixed. It is also linear with respect to `feq` ($f_i$) when $f_i == 0$ (`feq == 0`). And it is antisymmetric with respect to `feq` and $f_i$. Hence, the following tests:

In [5]:
using FactCheck: facts, @fact, context
facts("Behaviour of the single-relaxation time collision operator") do
    context("Linear vs τ⁻¹") do
        @fact 2collision(1, [1], [0]) => collision(2, [1], [0])
        @fact 3collision(1, [0], [1]) => collision(3, [0], [1])
    end
    context("Linear vs fᵢ (and feq)") do
        f = rand(Int8, 10)
        @fact 2collision(1, f, zeros(f)) => collision(1, 2f, zeros(f))
        @fact 3collision(1, zeros(f), f) => collision(1, zeros(f), 3f)
    end
    context("Anti-symmetric vs fᵢ and feq") do
        fᵢ, feq = rand(Int8, 10), rand(Int8, 10)
        @fact collision(1, fᵢ, feq) => -collision(1, feq, fᵢ)
    end
    context("Lock-down the sign") do
        @fact collision(1, [1:10], zeros(Int8, 10)) .< 0 => all
    end
end
nothing

Behaviour of the single-relaxation time collision operator
     - Linear vs τ⁻¹
     - Linear vs fᵢ (and feq)
     - Anti-symmetric vs fᵢ and feq
     - Lock-down the sign
6 facts verified.


Note the last fact-check in the code above. It isn't there as much to test as it is to make the code a bit more future proof. It merely records the chosen sign convention: does `collision` return $\tau^{-1}(f_{\text{eq}}-f_i)$ or $-\tau^{-1}(f_{\text{eq}}-f_i)$? That's very much implementation dependent. Inadvertently changing this convention when faffing with the collision code is an easy bug to introduce. Fortunately, it is an easy bug to ward against, so why not just do it? It's two minutes now vs two days sometime later.  

### Checking the flow of code and data -- a.k.a. Mocking

Once the internal bits of code are tested, it is time to test the scaffold that puts those bits together. Mocking, e.g. replacing complex  functions with fakes can be very useful at this juncture. It allows better separation of concerns when testing: bugs introduced in internal methods will not invalidate tests that replace them with Mocks. It makes for faster testing when the mocked code replaces computationally intensive routines. And most importantly, it  means focusing the tests of integration functions specifically on  what integration functions do: coordinate the functionalities of internal routines into a whole.  In the spirit of that latter remark, mocking is really about  analyzing the flow of data  and data transformations through a piece of code.  

This approach may look complicated at first. However, it truly is powerful and does simplify testing once one has wrapped one's head around it. The presentation that follows contains somewhat more boiler-plate than strictly necessary: at some point somebody will write a mocking framework for Julia and it will disappear.

Lets look at  an example. The code below  is  (a simplified version of) of the main  Lattice-Boltzmann kernel as it is  applied to each lattice site:

In [6]:
using LatBo: Simulation
using LatBo.LB: LocalKernel
function local_kernel(kernel::LocalKernel, sim::Simulation, site::Integer)
    const feq = equilibrium(sim.lattice, sim.populations[:, site])
    sim.populations[:, site] += collision(kernel.collision, sim.populations[:, site], feq)
    for direction in 1:length(sim.lattice)
        const to = neighbor_index(sim, site, direction)
        const streamer = kernel.streamers[sim.playground[to]]
        streaming(streamer, sim, site, to, direction)
    end
end

local_kernel (generic function with 1 method)

Roughly, the function proceeds for a given site as follows:
1. compute the number of particles for each population at equilibrium
1. collide the particles and update the number of particles
1. loop over all population/streaming direction
   1. figure out the index of the site that will be streamed to
   1. figure out the type of the site (fluid or boundary of some kind)
   1. stream the particles from this site to ... wherever
   
However, lets now forget about the physics and merely contemplate whether the flow of data goes as expected through the function, with the right calls. To do this we will redefine most of the inputs and functions. We will start by redefining the types, keeping only those attributes that are called explicitly in the function. 

In [7]:
using LatBo: Simulation
using LatBo.LB: Collision, Streaming, LocalKernel

type MockLattice end

type MockSimulation <: Simulation
    populations::Matrix
    playground::Matrix{Integer}
    lattice::MockLattice
end

type MockKernel <: LocalKernel
    collision :: Collision
    streamers :: Dict{Integer, Streaming}
end

type MockCollision <: Collision end
type MockStreamer <: Streaming end

As you can see, the definition of types are strictly limited to include the data that will be needed explicitly in the function. This is a bonus we get from applying this kind of test: we have pared down the inputs explicitly down to include only the moving parts. The test becomes a specification for `local_kernel`. It tells us exactly what piece of data is important, and what is not. Now we redefine the functions:

In [8]:
# first a function that stores the calls
global allcalls = Any[]
add_to_calls(args...) = push!(allcalls, args)

import LatBo.LB: equilibrium
import Base: length

function equilibrium(latt::MockLattice, fᵢ::Vector)
    add_to_calls(:equilibrium, latt, deepcopy(fᵢ))
    ones(fᵢ)
end
function collision(coll::MockCollision, fᵢ::Vector, feq::Vector)
    add_to_calls(:collision, coll, deepcopy(fᵢ), deepcopy(feq))
    ones(fᵢ)
end
function length(latt::MockLattice)
    add_to_calls(:length, latt)
    2
end
function neighbor_index(sim::MockSimulation, site::Integer, direction::Integer)
    add_to_calls(:neighbor_index, sim, site, direction)
    3
end
function streaming(
        streamer::MockStreamer, sim::MockSimulation,
        site::Integer, to::Integer, direction::Integer)
    add_to_calls(:streaming, streamer, sim, site, to, direction)
end

streaming (generic function with 1 method)

In [9]:
using FactCheck: facts, @fact, roughly, exactly
facts("Analyse data-flow through local kernel") do
    global allcalls
    empty!(allcalls)
    
    const site = 1
    const pop = 1. + randn(3, 10)
    playground = 42 * ones(Int, size(pop))
    playground[3] = 1
    sim = MockSimulation(deepcopy(pop), playground, MockLattice())
    kernel = MockKernel(MockCollision(), {1 => MockStreamer(), 42 => MockStreamer()})
    
    local_kernel(kernel, sim, site)
    
    # First computes equilibrium
    @fact allcalls[1][1] => :equilibrium
    # Check the lattice is the exact same as in sim (same object in memory)
    @fact allcalls[1][2] => exactly(sim.lattice)
    @fact allcalls[1][3] => roughly(pop[:, site])

    # Then computes collision
    @fact allcalls[2][1] => :collision
    @fact allcalls[2][2] => exactly(kernel.collision)
    @fact allcalls[2][3] => roughly(pop[:, site])
    # This is actually the return from the call to equilibrium
    @fact allcalls[2][4] => roughly(ones(pop[:, site]))
    
    # Then add result from collision to next population
    @fact sim.populations[:, site] => roughly(pop[:, site] + 1)
    # But other sites are untouched
    untouched = vcat(1:(site-1), (site+1):size(pop, 2))
    @fact sim.populations[:, untouched] => roughly(pop[:, untouched])
    
    # Get size of loop
    @fact allcalls[3][1] => :length
    @fact allcalls[3][2] => exactly(sim.lattice)
    
    # And run though loop twice
    direction = 1
    for i in 1:2:4
        # get index
        @fact allcalls[3 + i][1] => :neighbor_index
        @fact allcalls[3 + i][2] => exactly(sim)
        @fact allcalls[3 + i][3:end] => (site, direction)
        # then call streaming
        @fact allcalls[4 + i][1] => :streaming
        @fact allcalls[4 + i][2] => exactly(kernel.streamers[1])
        @fact allcalls[4 + i][3] => exactly(sim)
        @fact allcalls[4 + i][4:end] => (site, 3, direction)
        direction += 1
    end
end
nothing

Analyse data-flow through local kernel
25 facts verified.


This is fairly well decoupled from the rest of the code in LatBo. Yet it checks exactly the algorithm we would expect to take place in the local kernel: how the populations are transformed and which function calls are made. Strangely, there is no need to understand much of the underlying physics. Instead we place ourselves in a state of mind where we mimic the computer walking through the algorithm. Of course, if the algorithm does not recover the science it means to model, then this will not be the test to uncover such an error (although one could run the test using more physical data). Rather, it aims to ensure that we implemented exactly what we thought we were implementing. 